Import the needed libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
 # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'CKDWP5VJMC3VBZKCUEV4NUQHKRIQ4WL1HK0CHNGCGGY12SXL' # your Foursquare ID
CLIENT_SECRET = 'Q2W2QUMC5FQHZYOICKFXL0ZE1IJHFVRJ4YWAL3FAA4YNXIBN' # your Foursquare Secret
ACCESS_TOKEN = 'LNRTSWVI122MIVO3PRDURET2WMHQDNOTOJBCZFI1AOOI4TKB' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CKDWP5VJMC3VBZKCUEV4NUQHKRIQ4WL1HK0CHNGCGGY12SXL
CLIENT_SECRET:Q2W2QUMC5FQHZYOICKFXL0ZE1IJHFVRJ4YWAL3FAA4YNXIBN


In [3]:
from geopy.geocoders import Nominatim 

In order to define an instance of the geocoder, we need to define a user_agent. 
We will name our agent foursquare_agent, as shown below.

Suppose the case: in the polytechnic university of Catalonia (UPC) a congress is intended to be celebrated. I need to know if the university is a suitable place to celebrate the congress. 
So, first, we extract the geo data of the place.

In [4]:
address = 'C. Jordi Girona, 3208034 Barcelona Cataluña España'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.3877523 2.1147272


A suitable place needs hotels to locate the people attending the congress. Are there hotels near the university?

In [5]:
search_query = 'Hotel'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

hotels = results['response']['venues']


dataframe = json_normalize(hotels)
dataframe.head()

Hotel .... OK!


/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4bc5a24b0a30d13a94385a9c,AC Hotel Victoria Suites by Marriott,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1625146058,False,"Beltran I Ròzpide, 7-9",41.389324,2.121338,"[{'label': 'display', 'lat': 41.389324, 'lng':...",579,08034,ES,Barcelona,Cataluña,España,"[Beltran I Ròzpide, 7-9, 08034 Barcelona Catal...",NaN,NaN,NaN
1,4adcda4cf964a520b93f21e3,Gran Hotel Princesa Sofía,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1625146058,False,"Plaça de Pius XII, 4",41.386517,2.123076,"[{'label': 'display', 'lat': 41.38651744622853...",710,08028,ES,Barcelona,Cataluña,España,"[Plaça de Pius XII, 4 (Av. Diagonal), 08028 Ba...",Av. Diagonal,NaN,NaN
2,4bcc5f5baeaaeee1a1f23d6d,Hotel Sansi Pedralbes,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1625146058,False,"Av. Pearson, 1-3",41.393435,2.111722,"[{'label': 'display', 'lat': 41.39343506028381...",680,NaN,ES,Barcelona,Cataluña,España,"[Av. Pearson, 1-3, Barcelona Cataluña]",NaN,NaN,NaN
3,5d30def44318c90008b07cba,Hotel Sofia Barcelona - In the Unbound Collect...,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1625146058,False,Plaça De Pius Xii 4,41.385980,2.123314,"[{'label': 'display', 'lat': 41.38598, 'lng': ...",743,08028,ES,Barcelona,Cataluña,España,"[Plaça De Pius Xii 4, 08028 Barcelona Cataluña]",NaN,NaN,NaN
4,4e38aa161fc7e413a51a4788,Fairmont Hotel Rey Juan Carlos I (Hotel Rey Ju...,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1625146058,False,Av Diagonal 661 671,41.381291,2.109191,"[{'label': 'display', 'lat': 41.381291, 'lng':...",855,08028,ES,Barcelona,Cataluña,España,"[Av Diagonal 661 671, 08028 Barcelona Cataluña]",NaN,Les Corts,35398022


As we can see, there are some hotels near the university to perform the congress. Let's keep the interesting information for us.

In [10]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['hotels.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,AC Hotel Victoria Suites by Marriott,Hotel,"Beltran I Ròzpide, 7-9",41.389324,2.121338,"[{'label': 'display', 'lat': 41.389324, 'lng':...",579,08034,ES,Barcelona,Cataluña,España,"[Beltran I Ròzpide, 7-9, 08034 Barcelona Catal...",NaN,NaN,4bc5a24b0a30d13a94385a9c
1,Gran Hotel Princesa Sofía,Hotel,"Plaça de Pius XII, 4",41.386517,2.123076,"[{'label': 'display', 'lat': 41.38651744622853...",710,08028,ES,Barcelona,Cataluña,España,"[Plaça de Pius XII, 4 (Av. Diagonal), 08028 Ba...",Av. Diagonal,NaN,4adcda4cf964a520b93f21e3
2,Hotel Sansi Pedralbes,Hotel,"Av. Pearson, 1-3",41.393435,2.111722,"[{'label': 'display', 'lat': 41.39343506028381...",680,NaN,ES,Barcelona,Cataluña,España,"[Av. Pearson, 1-3, Barcelona Cataluña]",NaN,NaN,4bcc5f5baeaaeee1a1f23d6d
3,Hotel Sofia Barcelona - In the Unbound Collect...,Hotel,Plaça De Pius Xii 4,41.385980,2.123314,"[{'label': 'display', 'lat': 41.38598, 'lng': ...",743,08028,ES,Barcelona,Cataluña,España,"[Plaça De Pius Xii 4, 08028 Barcelona Cataluña]",NaN,NaN,5d30def44318c90008b07cba
4,Fairmont Hotel Rey Juan Carlos I (Hotel Rey Ju...,Hotel,Av Diagonal 661 671,41.381291,2.109191,"[{'label': 'display', 'lat': 41.381291, 'lng':...",855,08028,ES,Barcelona,Cataluña,España,"[Av Diagonal 661 671, 08028 Barcelona Cataluña]",NaN,Les Corts,4e38aa161fc7e413a51a4788


A list of the hotels that are near the university

In [7]:
dataframe_filtered.name

0                  AC Hotel Victoria Suites by Marriott
1                             Gran Hotel Princesa Sofía
2                                 Hotel Sansi Pedralbes
3     Hotel Sofia Barcelona - In the Unbound Collect...
4     Fairmont Hotel Rey Juan Carlos I (Hotel Rey Ju...
5                Gimnasio del Gran Hotel Princesa Sofía
6                        Hotel Catalonia Rigoletto ****
7     Pool Hotel Rey Juan Carlos I (Piscina Hotel Re...
8                                    We Barcelona Hotel
9                                  abba Garden hotel 4*
10                          Senator Barcelona Spa Hotel
11                                 Hotel Upper Diagonal
12                                         Hotel Arenas
13                               Hotel Ginosi Pedralbes
14                                Hotel Alguer Camp Nou
15                    Junior Suite @Gran Hotel Princesa
16                                       Hotel L'Alguer
17    Salon de Peluquería Hotel Rey Juan Carlos 

I display the hotels in a map

In [9]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

hotels_map

Let's explore the closest hotel

In [15]:
hotel_id = '4bc5a24b0a30d13a94385a9c' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(hotel_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'storeId', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4bc5a24b0a30d13a94385a9c',
 'name': 'AC Hotel Victoria Suites by Marriott',
 'contact': {'phone': '932069900',
  'formattedPhone': '932 06 99 00',
  'twitter': 'ac_hoteles'},
 'location': {'address': 'Beltran I Ròzpide, 7-9',
  'lat': 41.389324,
  'lng': 2.1213379,
  'labeledLatLngs': [{'label': 'display', 'lat': 41.389324, 'lng': 2.1213379}],
  'postalCode': '08034',
  'cc': 'ES',
  'city': 'Barcelona',
  'state': 'Cataluña',
  'country': 'España',
  'formattedAddress': ['Beltran I Ròzpide, 7-9', '08034 Barcelona Cataluña']},
 'canonicalUrl': 'https://foursquare.com/v/ac-hotel-victoria-suites-by-marriott/4bc5a24b0a30d13a94385a9c',
 'categories': [{'id': '4bf58dd8d48988d1fa931735',
   'name': 'Hotel',
   'pluralName': 'Hotels',
   'shortName': 'Hotel',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 14},
 'url': 'https://www.marriott.com/hotels/travel/bcnsu-ac-ho

In [16]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This hotel has not been rated yet.')

6.4


I am also interested to know if there is a good restaurant offer near the university.

In [23]:
search_query = 'Restaurant'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

restaurant = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(restaurant)
dataframe

Restaurant .... OK!


/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,5d7a03325f452e00086d9815,Híbrid Cafè Restaurant,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1625147149,False,"Travessera de les Corts, 38",41.377148,2.120327,"[{'label': 'display', 'lat': 41.377148, 'lng':...",1269,08903,ES,Hospitalet de Llobregat,Katalánsko,España,"[Travessera de les Corts, 38, 08903 Hospitalet...",NaN,NaN
1,4fcf5b20e4b0a254617d09d8,Restaurant Cipri,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1625147149,False,NaN,41.378139,2.118414,"[{'label': 'display', 'lat': 41.37813887277589...",1113,NaN,ES,NaN,NaN,España,NaN,NaN,NaN
2,4bf3e3233f86c9b6b491b21c,Restaurant Julivert Meu,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",v-1625147149,False,NaN,41.387322,2.113345,"[{'label': 'display', 'lat': 41.38732191687644...",125,NaN,ES,NaN,NaN,España,NaN,NaN,NaN
3,4bbe09a78ec3d13aa5ce1c28,Restaurant Paradís,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1625147149,False,NaN,41.387590,2.112481,"[{'label': 'display', 'lat': 41.38759016990661...",188,NaN,ES,NaN,NaN,España,NaN,NaN,NaN
4,4b7d96acf964a52075c82fe3,Malam Restaurante,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1625147149,False,Avda. Doctor Marañon 17,41.381298,2.113859,"[{'label': 'display', 'lat': 41.381298, 'lng':...",722,NaN,ES,Barcelona,Cataluña,España,"[Avda. Doctor Marañon 17, Barcelona Cataluña]",NaN,NaN
5,4adcda55f964a520084321e3,Restaurante Tritón,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",v-1625147149,False,"Carrer de L´Alfambra, 16",41.386673,2.112519,"[{'label': 'display', 'lat': 41.386672582, 'ln...",220,08034,ES,Barcelona,Cataluña,España,"[Carrer de L´Alfambra, 16, 08034 Barcelona Cat...",NaN,NaN
6,4f3e75d0e4b00c683d521fb6,Bar Restaurant Les Corts,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",v-1625147149,False,Pintor Tapiró 45,41.378039,2.118198,"[{'label': 'display', 'lat': 41.37803888367190...",1119,08014,ES,Barcelona,Cataluña,España,"[Pintor Tapiró 45, 08014 Barcelona Cataluña]",NaN,NaN
7,598f6e46419a9e61647269f3,IMPAR Restaurant,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",v-1625147149,False,NaN,41.386454,2.123293,"[{'label': 'display', 'lat': 41.38645433824837...",729,08028,ES,Barcelona,Cataluña,España,[08028 Barcelona Cataluña],NaN,NaN
8,508aa74be4b0c56e748ad085,Eurest Cafeteria restaurant,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1625147149,False,NaN,41.391187,2.123485,"[{'label': 'display', 'lat': 41.39118678487992...",825,NaN,ES,NaN,NaN,España,NaN,NaN,NaN
9,4f6b2bc2e4b0454e85011bda,Polo Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1625147149,False,NaN,41.381416,2.109185,"[{'label': 'display', 'lat': 41.38141628721880...",843,NaN,ES,NaN,NaN,España,NaN,NaN,NaN


In [24]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['hotels.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,crossStreet,id
0,Híbrid Cafè Restaurant,Café,"Travessera de les Corts, 38",41.377148,2.120327,"[{'label': 'display', 'lat': 41.377148, 'lng':...",1269,08903,ES,Hospitalet de Llobregat,Katalánsko,España,"[Travessera de les Corts, 38, 08903 Hospitalet...",NaN,NaN,5d7a03325f452e00086d9815
1,Restaurant Cipri,Restaurant,NaN,41.378139,2.118414,"[{'label': 'display', 'lat': 41.37813887277589...",1113,NaN,ES,NaN,NaN,España,NaN,NaN,NaN,4fcf5b20e4b0a254617d09d8
2,Restaurant Julivert Meu,Mediterranean Restaurant,NaN,41.387322,2.113345,"[{'label': 'display', 'lat': 41.38732191687644...",125,NaN,ES,NaN,NaN,España,NaN,NaN,NaN,4bf3e3233f86c9b6b491b21c
3,Restaurant Paradís,Restaurant,NaN,41.387590,2.112481,"[{'label': 'display', 'lat': 41.38759016990661...",188,NaN,ES,NaN,NaN,España,NaN,NaN,NaN,4bbe09a78ec3d13aa5ce1c28
4,Malam Restaurante,Restaurant,Avda. Doctor Marañon 17,41.381298,2.113859,"[{'label': 'display', 'lat': 41.381298, 'lng':...",722,NaN,ES,Barcelona,Cataluña,España,"[Avda. Doctor Marañon 17, Barcelona Cataluña]",NaN,NaN,4b7d96acf964a52075c82fe3


In [25]:
restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=13) 


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(restaurant_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(restaurant_map)

restaurant_map

Finally, some participants may want to see the city. Good connections with the rest of the city are completelly needed.

In [27]:
search_query = 'Bus'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
results

restaurant = results['response']['venues']

dataframe = json_normalize(restaurant)
dataframe

Bus .... OK!


/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.city,location.state,location.formattedAddress,location.crossStreet,location.postalCode
0,4defd84045dd29e268a91ff4,Bus L79,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.386307,2.119473,"[{'label': 'display', 'lat': 41.38630668074690...",427,ES,España,NaN,NaN,NaN,NaN,NaN,NaN
1,4ddfe31ae4cde756db3ee48f,Parada Bus Av. Diagonal-Pl. Reina Mª Cristina,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1625147375,False,41.388062,2.127387,"[{'label': 'display', 'lat': 41.38806180022258...",1057,ES,España,Av. Diagonal,Barcelona,Cataluña,"[Av. Diagonal, Barcelona Cataluña]",NaN,NaN
2,4ed9964429c2b912277b135a,Parada Bus 1713,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.376882,2.119324,"[{'label': 'display', 'lat': 41.37688182482336...",1269,ES,España,Travessera De Les Corts,Hospitalet de Llobregat,Cataluña,"[Travessera De Les Corts (Carrer Del Danubi), ...",Carrer Del Danubi,NaN
3,4e53be76a8097c30ee82c934,Parada Bus JM y EP1,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.384624,2.110877,"[{'label': 'display', 'lat': 41.38462443610918...",473,ES,España,NaN,NaN,NaN,NaN,NaN,NaN
4,506ac0cfe4b0ac63fc655d38,Parada Bus 2586,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.385729,2.114699,"[{'label': 'display', 'lat': 41.38572885048456...",225,ES,España,Carrer Coronel Valenzuela,Barcelona,Cataluña,[Carrer Coronel Valenzuela (Avinguda Diagonal)...,Avinguda Diagonal,NaN
5,4d6be6b93a6bf04df017597d,Parada Bus Av. Diagonal-Palau Reial,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.385833,2.117757,"[{'label': 'display', 'lat': 41.38583347904427...",331,ES,España,Av. Diagonal,Barcelona,Cataluña,"[Av. Diagonal, Barcelona Cataluña]",NaN,NaN
6,528ca2d1498ef67bbda7db9e,TMB Bus V3,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.391027,2.118366,"[{'label': 'display', 'lat': 41.39102715774534...",474,ES,España,NaN,NaN,NaN,NaN,NaN,NaN
7,4e9db4a5d3e31270c904bce8,Parada 1863 Bus 54,"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",v-1625147375,False,41.389445,2.112653,"[{'label': 'display', 'lat': 41.38944515528710...",255,ES,España,carrer de sor Eulàlia d'anzizu,Barcelona,Cataluña,[carrer de sor Eulàlia d'anzizu (carrer del gr...,carrer del gran capità,NaN
8,5644329a498eef663c0131fb,Parada Bus Hillsa Maria Cristina,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1625147375,False,41.385985,2.112615,"[{'label': 'display', 'lat': 41.38598490892826...",264,ES,España,NaN,NaN,NaN,NaN,NaN,NaN
9,4f1ebbfde4b07f86b54b3d7d,Bus 1413,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",v-1625147375,False,41.390462,2.118783,"[{'label': 'display', 'lat': 41.39046159736407...",453,ES,España,NaN,Barcelona,Cataluña,[Barcelona Cataluña],NaN,NaN


In [28]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['hotels.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,city,state,formattedAddress,crossStreet,postalCode,id
0,Bus L79,Bus Line,41.386307,2.119473,"[{'label': 'display', 'lat': 41.38630668074690...",427,ES,España,NaN,NaN,NaN,NaN,NaN,NaN,4defd84045dd29e268a91ff4
1,Parada Bus Av. Diagonal-Pl. Reina Mª Cristina,Bus Stop,41.388062,2.127387,"[{'label': 'display', 'lat': 41.38806180022258...",1057,ES,España,Av. Diagonal,Barcelona,Cataluña,"[Av. Diagonal, Barcelona Cataluña]",NaN,NaN,4ddfe31ae4cde756db3ee48f
2,Parada Bus 1713,Bus Line,41.376882,2.119324,"[{'label': 'display', 'lat': 41.37688182482336...",1269,ES,España,Travessera De Les Corts,Hospitalet de Llobregat,Cataluña,"[Travessera De Les Corts (Carrer Del Danubi), ...",Carrer Del Danubi,NaN,4ed9964429c2b912277b135a
3,Parada Bus JM y EP1,Bus Line,41.384624,2.110877,"[{'label': 'display', 'lat': 41.38462443610918...",473,ES,España,NaN,NaN,NaN,NaN,NaN,NaN,4e53be76a8097c30ee82c934
4,Parada Bus 2586,Bus Line,41.385729,2.114699,"[{'label': 'display', 'lat': 41.38572885048456...",225,ES,España,Carrer Coronel Valenzuela,Barcelona,Cataluña,[Carrer Coronel Valenzuela (Avinguda Diagonal)...,Avinguda Diagonal,NaN,506ac0cfe4b0ac63fc655d38


In [29]:
bus_map = folium.Map(location=[latitude, longitude], zoom_start=13)


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(bus_map)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(bus_map)

bus_map